In [6]:
# Instalando blibiotecas
# !pip install pandas
# !pip install scikit-learn
# !pip install numpy
# !pip install xgboost
# !pip install lightgbm
# !pip install catboost

In [8]:
import pandas as pd
import os

# Define o caminho para a pasta que contém as pastas de ano
base_folder_path = './dataset/'

# Cria uma lista para armazenar os DataFrames de cada arquivo
df_list = []

# Lista todos os itens na pasta base
all_items = os.listdir(base_folder_path)

# Filtra apenas as pastas (que devem ser os anos)
year_folders = [d for d in all_items if os.path.isdir(os.path.join(base_folder_path, d))]

# Itera sobre as pastas de ano
for year_folder in year_folders:
    year_path = os.path.join(base_folder_path, year_folder)

    # Extrai o ano do nome da pasta
    try:
        year_from_folder = int(year_folder)
    except ValueError:
        print(f"Ignorando pasta '{year_folder}' pois o nome não é um ano válido.")
        continue # Skip to the next folder if the name is not a valid year

    # Lista todos os arquivos na pasta do ano
    year_files = os.listdir(year_path)

    # Filtra apenas os arquivos CSV dentro da pasta do ano
    csv_files = [f for f in year_files if f.endswith('.CSV')]

    # Itera sobre os arquivos CSV dentro da pasta do ano
    for csv_file in csv_files:
        file_path = os.path.join(year_path, csv_file)
        try:
            df = pd.read_csv(file_path, sep=';')  # Specify semicolon delimiter

            # Use the year from the folder
            df['ANO'] = year_from_folder

            # Convert 'MES' to string before removing comma and converting to numeric
            df['MES'] = df['MES'].astype(str).str.replace(',', '', regex=False) # Remove comma
            df['MES'] = pd.to_numeric(df['MES'], errors='coerce').fillna(0).astype(int)

            # Create a simple date format (YYYY-MM) - assuming day is not important or always 1
            df['DATA'] = pd.to_datetime(df['ANO'].astype(str) + '-' + df['MES'].astype(str), format='%Y-%m', errors='coerce')

            # Extract year and month into new columns
            df['ANO'] = df['DATA'].dt.year
            df['MES'] = df['DATA'].dt.month

            # Drop the original date columns
            df = df.drop(columns=['nr_ano_referencia', 'nr_mes_referencia', 'DATA'], errors='ignore')

            df_list.append(df)
        except Exception as e:
            print(f"Erro ao ler o arquivo {csv_file} na pasta {year_folder}: {e}")

# Concatena todos os DataFrames na lista em um único DataFrame
if df_list:
    main_df = pd.concat(df_list, ignore_index=True)
    print("DataFrame combinado criado com sucesso!")

    # Reorder columns
    cols = main_df.columns.tolist()
    cols.remove('MES')
    cols.remove('ANO')
    cols.insert(cols.index('ASSENTOS') + 1, 'MES')
    cols.insert(cols.index('ASSENTOS') + 2, 'ANO')
    main_df = main_df[cols]

    print(main_df.head())
else:
    print("Nenhum arquivo CSV encontrado ou lido nas pastas de ano.")

Ignorando pasta 'backup' pois o nome não é um ano válido.
DataFrame combinado criado com sucesso!
  EMPRESA ORIGEM DESTINO   TARIFA  ASSENTOS  MES   ANO
0     ABJ   SBSV    SIRI   650,00        17    1  2023
1     ABJ   SBSV    SIRI   850,00        23    1  2023
2     ABJ   SBSV    SIRI  1050,00         6    1  2023
3     ABJ   SBSV    SIRI  1250,00         1    1  2023
4     ABJ   SBSV    SNCL   450,00         1    1  2023


In [9]:
# Create a dictionary with the mapping of abbreviations to full names
empresa_mapping = {
    'ABJ': 'ATA - AEROTÁXI ABAETÉ LTDA.',
    'AZU': 'AZUL LINHAS AÉREAS BRASILEIRAS S/A',
    'GLO': 'GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.)',
    'TAM': 'TAM LINHAS AÉREAS S.A.',
    'CQB': 'APUÍ TÁXI AÉREO S/A'
}

# Replace the abbreviations in the 'EMPRESA' column
main_df['EMPRESA'] = main_df['EMPRESA'].replace(empresa_mapping)

# Display the updated unique values in the 'EMPRESA' column to verify
print("Unique values in 'EMPRESA' column after replacement:")
display(main_df['EMPRESA'].unique())

print(main_df.head())

Unique values in 'EMPRESA' column after replacement:


array(['ATA - AEROTÁXI ABAETÉ LTDA.',
       'AZUL LINHAS AÉREAS BRASILEIRAS S/A',
       'GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.)', 'PTB',
       'TAM LINHAS AÉREAS S.A.', 'APUÍ TÁXI AÉREO S/A'], dtype=object)

                       EMPRESA ORIGEM DESTINO   TARIFA  ASSENTOS  MES   ANO
0  ATA - AEROTÁXI ABAETÉ LTDA.   SBSV    SIRI   650,00        17    1  2023
1  ATA - AEROTÁXI ABAETÉ LTDA.   SBSV    SIRI   850,00        23    1  2023
2  ATA - AEROTÁXI ABAETÉ LTDA.   SBSV    SIRI  1050,00         6    1  2023
3  ATA - AEROTÁXI ABAETÉ LTDA.   SBSV    SIRI  1250,00         1    1  2023
4  ATA - AEROTÁXI ABAETÉ LTDA.   SBSV    SNCL   450,00         1    1  2023


In [10]:
csv_file_path = './dataset/cadastro-de-aerodromos-civis-publicos.csv'

try:
    airport_df = pd.read_csv(csv_file_path, sep=';')

    # Include the 'UF' column
    airport_df = airport_df[['CÓDIGO OACI', 'MUNICÍPIO ATENDIDO', 'UF']]

    print("New DataFrame created with 'CÓDIGO OACI', 'MUNICÍPIO ATENDIDO', and 'UF' columns:")
    display(airport_df.head())

except FileNotFoundError:
    print(f"Error: The file was not found at {csv_file_path}")
except KeyError:
    print("Error: 'CÓDIGO OACI', 'MUNICÍPIO ATENDIDO', or 'UF' columns not found in the CSV.")
except Exception as e:
    print(f"An error occurred: {e}")

New DataFrame created with 'CÓDIGO OACI', 'MUNICÍPIO ATENDIDO', and 'UF' columns:


,CÓDIGO OACI,MUNICÍPIO ATENDIDO,UF
0,SBAA,CONCEIÇÃO DO ARAGUAIA,PA
1,SBAE,BAURU,SP
2,SBAQ,ARARAQUARA,SP
3,SBAR,ARACAJU,SE
4,SBAT,ALTA FLORESTA,MT


In [ ]:
import pandas as pd

# Passo 1: Limpar e padronizar os dados
main_df_clean = main_df.copy()
airport_df_clean = airport_df.copy()

# Limpar espaços e converter para maiúsculas
main_df_clean['ORIGEM'] = main_df_clean['ORIGEM'].str.strip().str.upper()
main_df_clean['DESTINO'] = main_df_clean['DESTINO'].str.strip().str.upper()
airport_df_clean['CÓDIGO OACI'] = airport_df_clean['CÓDIGO OACI'].str.strip().str.upper()

if 'CÓDIGO IATA' in airport_df_clean.columns:
    airport_df_clean['CÓDIGO IATA'] = airport_df_clean['CÓDIGO IATA'].str.strip().str.upper()

# Passo 2: Merge para ORIGEM (que já estava funcionando)
merged_origin_df = pd.merge(
    main_df_clean,
    airport_df_clean,
    left_on='ORIGEM',
    right_on='CÓDIGO OACI',
    how='left'
)

# Renomear e reorganizar ORIGEM
merged_origin_df = merged_origin_df.rename(columns={'MUNICÍPIO ATENDIDO': 'MUNICIPIO_ORIGEM'})
merged_origin_df = merged_origin_df.drop(columns=['CÓDIGO OACI', 'UF', 'ORIGEM'], errors='ignore')
merged_origin_df = merged_origin_df.rename(columns={'MUNICIPIO_ORIGEM': 'ORIGEM'})

# Reordenar colunas
cols = merged_origin_df.columns.tolist()
cols.remove('ORIGEM')
cols.insert(cols.index('EMPRESA') + 1, 'ORIGEM')
merged_origin_df = merged_origin_df[cols]

# Passo 3: Merge para DESTINO com verificação
print("Verificando merge para DESTINO...")

# Tentar merge com OACI
merged_final_df = pd.merge(
    merged_origin_df,
    airport_df_clean,
    left_on='DESTINO',
    right_on='CÓDIGO OACI',
    how='left'
)

# Verificar resultado
matches_destino = merged_final_df['MUNICÍPIO ATENDIDO'].notna().sum()
print(f"Matches encontrados para DESTINO: {matches_destino}/{len(merged_final_df)}")

# Se não encontrou matches, mostrar exemplos problemáticos
if matches_destino == 0:
    print("\nCódigos problemáticos em DESTINO:")
    problematic_codes = merged_origin_df['DESTINO'].unique()[:10]
    for code in problematic_codes:
        print(f"  '{code}' -> Existe em airport_df? {code in airport_df_clean['CÓDIGO OACI'].values}")

# Continuar com o processamento independente do resultado
merged_final_df = merged_final_df.rename(columns={'MUNICÍPIO ATENDIDO': 'MUNICIPIO_DESTINO'})
merged_final_df = merged_final_df.drop(columns=['CÓDIGO OACI', 'UF', 'DESTINO'], errors='ignore')
merged_final_df = merged_final_df.rename(columns={'MUNICIPIO_DESTINO': 'DESTINO'})

# Reordenar colunas
cols = merged_final_df.columns.tolist()
cols.remove('DESTINO')
cols.insert(cols.index('ORIGEM') + 1, 'DESTINO')
merged_final_df = merged_final_df[cols]

# Remover linhas onde ORIGEM ou DESTINO são nulos ou NaN
rows_before = len(merged_final_df)
merged_final_df = merged_final_df.dropna(subset=['ORIGEM', 'DESTINO'])
rows_after = len(merged_final_df)
rows_removed = rows_before - rows_after
print(f"\nLinhas removidas com ORIGEM ou DESTINO nulos: {rows_removed} ({(rows_removed/rows_before)*100:.2f}% do total)")

# Display the head of the final dataframe to verify
print("\nDataFrame after merging destination airport municipality and removing null values:")
display(merged_final_df.head())
print(f"\nShape do DataFrame final: {merged_final_df.shape}")

In [ ]:
# Define o caminho e nome do arquivo para exportar
output_path = './validacao.csv'

# Exporta o DataFrame para CSV
try:
    merged_final_df.to_csv(output_path, index=False)
    print(f"DataFrame exportado com sucesso para {output_path}")
except Exception as e:
    print(f"Erro ao exportar o DataFrame: {e}")

DataFrame exportado com sucesso para ./validacao.csv


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Otimizar tipos de dados para reduzir uso de memória
merged_final_df['TARIFA'] = merged_final_df['TARIFA'].astype(str).str.replace(',', '.').astype('float32')
merged_final_df['MES'] = merged_final_df['MES'].astype('int8')
merged_final_df['ANO'] = merged_final_df['ANO'].astype('int16')
merged_final_df['ASSENTOS'] = merged_final_df['ASSENTOS'].astype('int32')

# Verificar cardinalidade das colunas categóricas
print("Número de valores únicos em cada coluna:")
for col in ['EMPRESA', 'ORIGEM', 'DESTINO']:
    print(f"{col}: {merged_final_df[col].nunique()} valores únicos")

# Variável alvo
y = merged_final_df['ASSENTOS'].values

# Criar cópia das features numéricas com tipos otimizados
X = merged_final_df[['TARIFA', 'MES', 'ANO']].copy()

# Aplicar Label Encoding para cada coluna categórica
encoders = {}
for col in ['EMPRESA', 'ORIGEM', 'DESTINO']:
    le = LabelEncoder()
    X[col] = le.fit_transform(merged_final_df[col].astype(str))
    encoders[col] = le
    X[col] = X[col].astype('int32')  # Otimizar tipo após encoding

print("\nShape do DataFrame X após encoding:", X.shape)
print("\nUso de memória otimizado para as features.")

Número de valores únicos em cada coluna:
EMPRESA: 5 valores únicos
ORIGEM: 166 valores únicos
DESTINO: 165 valores únicos

Shape do DataFrame X após encoding: (17765412, 6)

Uso de memória otimizado para as features.


In [ ]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Dividir dados com estratificação para melhor distribuição
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configurar validação cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return r2, rmse

print("Dados divididos e validação cruzada configurada.")

Dados divididos e validação cruzada configurada.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

r2_lr = r2_score(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
print(f"Regressão Linear - R²: {r2_lr:.4f}, RMSE: {rmse_lr:.2f}")

Regressão Linear - R²: 0.0125, RMSE: 18.86


In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Configurar XGBoost com parâmetros otimizados
xgb_params = {
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'learning_rate': 0.05,
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1,
    'early_stopping_rounds': 20
}

# Treinar XGBoost
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

y_pred_xgb = xgb_model.predict(X_test)
r2_xgb = r2_score(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print(f"XGBoost - R²: {r2_xgb:.4f}, RMSE: {rmse_xgb:.2f}")

XGBoost - R²: 0.0477, RMSE: 18.52


In [ ]:
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Configurar LightGBM com parâmetros otimizados
lgb_params = {
    'objective': 'regression',
    'n_estimators': 100,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'random_state': 42,
    'n_jobs': -1,
    'early_stopping_round': 20,
    'verbose': -1
}

# Treinar LightGBM
lgb_model = lgb.LGBMRegressor(**lgb_params)
lgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y_pred_lgbm = lgb_model.predict(X_test)
r2_lgbm = r2_score(y_test, y_pred_lgbm)
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))
print(f"LightGBM - R²: {r2_lgbm:.4f}, RMSE: {rmse_lgbm:.2f}")

LightGBM - R²: 0.0491, RMSE: 18.51


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Treinar Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

r2_gb = r2_score(y_test, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
print(f"Gradient Boosting - R²: {r2_gb:.4f}, RMSE: {rmse_gb:.2f}")

Gradient Boosting - R²: 0.0379, RMSE: 18.62


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Treinar AdaBoost
ada_model = AdaBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
ada_model.fit(X_train, y_train)
y_pred_ada = ada_model.predict(X_test)

r2_ada = r2_score(y_test, y_pred_ada)
rmse_ada = np.sqrt(mean_squared_error(y_test, y_pred_ada))
print(f"AdaBoost - R²: {r2_ada:.4f}, RMSE: {rmse_ada:.2f}")

KeyboardInterrupt: 